#### 0601_Lab   
Due date
- 25 Oct 2023 for Sections A and C   
- 26 Oct 2023 for Section B

1. Create tables for MLAs of the current and the previous assemblies of Karnataka.  
    Name them as MLA_2018 and MLA_2023.  
    Columns AC_Num Int Primary Key, MLA_Name Text, Party TEXT.   
    Insert values   

In [1]:
import pandas as pd
import os
import sqlite3 as lite
from glob import glob

dbpath = '0601_Elections_Lab.sqlite'
con = lite.connect(dbpath)
cursor = con.cursor()

In [2]:
df = pd.read_csv('/Users/reethu/coding/college/DBMS/Databases/KA_MLA_2018_to_2023.csv')

In [3]:
#con.execute('DROP TABLE MLA_2018')
sql = """ CREATE TABLE MLA_2018(
        AC_Num Integer Primary Key,
        MLA_Name Text,
        Party Text)
      """


con.execute(sql)
con.commit()


In [4]:
df.to_sql('MLA_2018_temp',con, if_exists='replace',index=False)
pd.read_sql("Select * from MLA_2018_temp",con)

,AC_Num,Constituency,MLA_Name,Party
0,1,Nippani,Shashikala Annasaheb Jolle,BJP
1,2,Chikkodi-Sadalga,Ganesh Hukkeri,INC
2,3,Athani,Mahesh Kumathalli,BJP
3,4,Kagwad,Srimant Patil,BJP
4,5,Kudachi,P. Rajeev,BJP
...,...,...,...,...
219,220,T Narasipura,Ashvin Kumar M.,JDS
220,221,Hanur,R. Narendra,INC
221,222,Kollegal,N. Mahesh,BJP
222,223,Chamrajanagar,C. Puttarangashetty,INC


In [5]:
print(pd.read_sql(f"PRAGMA table_info (MLA_2018)",con))

   cid      name     type  notnull dflt_value  pk
0    0    AC_Num  INTEGER        0       None   1
1    1  MLA_Name     TEXT        0       None   0
2    2     Party     TEXT        0       None   0


In [6]:
sql = """Insert into MLA_2018(AC_Num, MLA_Name, Party)
        Select AC_Num, MLA_Name, Party from MLA_2018_temp """

con.execute(sql)

In [7]:
sql=""" Drop table if exists MLA_2018_temp"""
con.execute(sql)
con.commit()

In [8]:
pd.read_sql("Select * from MLA_2018",con)

,AC_Num,MLA_Name,Party
0,1,Shashikala Annasaheb Jolle,BJP
1,2,Ganesh Hukkeri,INC
2,3,Mahesh Kumathalli,BJP
3,4,Srimant Patil,BJP
4,5,P. Rajeev,BJP
...,...,...,...
219,220,Ashvin Kumar M.,JDS
220,221,R. Narendra,INC
221,222,N. Mahesh,BJP
222,223,C. Puttarangashetty,INC


In [9]:
#con.execute('DROP TABLE MLA_2023')
sql = """ CREATE TABLE MLA_2023(
        AC_Num Integer Primary Key,
        MLA_Name Text,
        Party Text)
      """

con.execute(sql)
con.commit()

In [10]:
df1 = pd.read_excel('/Users/reethu/coding/college/DBMS/Databases/KA2023_Winners.xlsx')

In [11]:
df1.to_sql('MLA_2023_temp',con, if_exists='replace',index=False)
pd.read_sql("Select * from MLA_2023_temp",con)

,Sl No.,Constituency Name,Winners,Party
0,1,Nippani,Jolle Shashikala Annasaheb,BJP
1,2,Chikkodi-Sadalga,Ganesh Prakash Hukkeri,Congress
2,3,Athani,Laxman Savadi,Congress
3,4,Kagwad,Bharamgouda Alagouda Kage,Congress
4,5,Kudachi (SC),Mahendra Kallappa Tammannavar,Congress
...,...,...,...,...
219,220,T. Narasipur (SC),Dr H C Mahadevappa,Congress
220,221,Hanur,M R Manjunath,JD(S)
221,222,Kollegal (SC),A R Krishnamurthy,Congress
222,223,Chamarajanagar,C Puttarangashetty,Congress


In [12]:
sql = """ Alter table MLA_2023_temp RENAME COLUMN "Sl No." to AC_Num """
con.execute(sql)
con.commit()

In [13]:
sql = """Insert into MLA_2023(AC_Num, MLA_Name, Party)
        Select CAST(AC_Num as INTEGER), Winners, Party from MLA_2023_temp """

con.execute(sql)

In [14]:
pd.read_sql("Select * from MLA_2023",con)

,AC_Num,MLA_Name,Party
0,0,Ravi Subramanya L A,BJP
1,1,Jolle Shashikala Annasaheb,BJP
2,2,Ganesh Prakash Hukkeri,Congress
3,3,Laxman Savadi,Congress
4,4,Bharamgouda Alagouda Kage,Congress
...,...,...,...
219,220,Dr H C Mahadevappa,Congress
220,221,M R Manjunath,JD(S)
221,222,A R Krishnamurthy,Congress
222,223,C Puttarangashetty,Congress


In [15]:
sql = """ UPDATE MLA_2023 set AC_Num = 170 where AC_Num = 0 """
con.execute(sql)
con.commit()

In [16]:
sql=""" Drop table if exists MLA_2023_temp"""
con.execute(sql)
con.commit()

2. Create a new table MLA_KA making union of the above two tables.  
    Before making the union, correct MLA names of the same person written differently in the two tables.  
    Justify the resulting row count.   

In [17]:

pd.read_sql("SELECT Distinct(party) from MLA_2023",con)

,Party
0,BJP
1,Congress
2,JD(S)
3,KRPP
4,Independent
5,SKP-Congress


In [18]:
sql1 = "UPDATE MLA_2023 set Party = 'INC' where Party = 'Congress' "

sql2 = "UPDATE MLA_2023 set Party = 'JDS' where Party= 'JD(S)' "

#con.execute(sql1)
con.execute(sql2)
con.commit()

In [19]:
for i in range(1,225):
  lis = con.execute(f"Select MLA_Name, Party from MLA_2018 where AC_Num={i}").fetchall()
  name, par =lis[0]

  sql = (f""" UPDATE MLA_2023
          SET MLA_NAME = '{name}'
          WHERE AC_Num = {i} AND Party = '{par}' """)
  con.execute(sql)

In [20]:
con.commit()

In [21]:
sql = """
SELECT MLA_Name, Party FROM MLA_2023
UNION
SELECT MLA_Name, Party FROM MLA_2018
"""

pd.read_sql(sql, con)

,MLA_Name,Party
0,A B Ramesha Bandisiddegowda,Congress
1,A Manju,JDS
2,A R Krishnamurthy,Congress
3,A S Ponnanna,Congress
4,A. C. Srinivasa,Congress
...,...,...
384,Vithal Somanna Halagekar,BJP
385,Y. V. Patil,INC
386,Yashavantaraygoud. Vittalagoud. Patil,Congress
387,Yathindra S.,INC


In [22]:
#con.execute('DROP TABLE MLA_KA')
sql = """
CREATE TABLE MLA_KA  AS
    SELECT MLA_Name, Party FROM MLA_2023
    UNION
    SELECT MLA_Name, Party FROM MLA_2018
"""
con.execute(sql)

con.commit()

In [23]:
sql = """Delete from MLA_KA where MLA_Name = 'Vacant'"""
con.execute(sql)
con.commit()

In [24]:
pd.read_sql("Select * from MLA_KA",con)

,MLA_Name,Party
0,A B Ramesha Bandisiddegowda,Congress
1,A Manju,JDS
2,A R Krishnamurthy,Congress
3,A S Ponnanna,Congress
4,A. C. Srinivasa,Congress
...,...,...
383,Vithal Somanna Halagekar,BJP
384,Y. V. Patil,INC
385,Yashavantaraygoud. Vittalagoud. Patil,Congress
386,Yathindra S.,INC


3. Use tables KA_Constituencies, MLA_2018, and MLA_2023.  
    Setting appropriate relationships among these tables, print   
    AC_Num, AC_Name, MLA in 2018, Party, MLA in 2023, and Party


In [25]:
df.to_sql('Const_temp',con, if_exists='replace',index=False)
pd.read_sql("Select * from Const_temp",con)

,AC_Num,Constituency,MLA_Name,Party
0,1,Nippani,Shashikala Annasaheb Jolle,BJP
1,2,Chikkodi-Sadalga,Ganesh Hukkeri,INC
2,3,Athani,Mahesh Kumathalli,BJP
3,4,Kagwad,Srimant Patil,BJP
4,5,Kudachi,P. Rajeev,BJP
...,...,...,...,...
219,220,T Narasipura,Ashvin Kumar M.,JDS
220,221,Hanur,R. Narendra,INC
221,222,Kollegal,N. Mahesh,BJP
222,223,Chamrajanagar,C. Puttarangashetty,INC


In [26]:
sql = """
   CREATE TABLE KA_Constituencies(
     AC_Num int Primary Key,
     AC_Name Text
   )
"""
con.execute("DROP TABLE IF EXISTS KA_Constituencies")
con.execute(sql)
con.commit()

In [27]:
sql = """
  INSERT INTO KA_Constituencies
  SELECT AC_Num, Constituency FROM Const_temp
"""
con.execute(sql)
con.commit()

In [28]:
pd.read_sql("Select * from KA_Constituencies",con)

,AC_Num,AC_Name
0,1,Nippani
1,2,Chikkodi-Sadalga
2,3,Athani
3,4,Kagwad
4,5,Kudachi
...,...,...
219,220,T Narasipura
220,221,Hanur
221,222,Kollegal
222,223,Chamrajanagar


In [29]:
sql=""" Drop table if exists Const_temp"""
con.execute(sql)
con.commit()

In [30]:
sql = """ Select a.AC_Num, a.AC_Name,
      b.MLA_Name as MLA_2018, b.Party as Party_2018,
      c.MLA_Name as MLA_2023, c.Party as Party_2023
      from KA_Constituencies as a, MLA_2018 as b, MLA_2023 as c
      where a.AC_Num=b.AC_Num
      AND a.AC_Num=c.AC_Num """

pd.read_sql(sql,con)

,AC_Num,AC_Name,MLA_2018,Party_2018,MLA_2023,Party_2023
0,1,Nippani,Shashikala Annasaheb Jolle,BJP,Shashikala Annasaheb Jolle,BJP
1,2,Chikkodi-Sadalga,Ganesh Hukkeri,INC,Ganesh Prakash Hukkeri,Congress
2,3,Athani,Mahesh Kumathalli,BJP,Laxman Savadi,Congress
3,4,Kagwad,Srimant Patil,BJP,Bharamgouda Alagouda Kage,Congress
4,5,Kudachi,P. Rajeev,BJP,Mahendra Kallappa Tammannavar,Congress
...,...,...,...,...,...,...
219,220,T Narasipura,Ashvin Kumar M.,JDS,Dr H C Mahadevappa,Congress
220,221,Hanur,R. Narendra,INC,M R Manjunath,JDS
221,222,Kollegal,N. Mahesh,BJP,A R Krishnamurthy,Congress
222,223,Chamrajanagar,C. Puttarangashetty,INC,C Puttarangashetty,Congress


4. Create table MLA_Rel and insert the above values


In [31]:
sql = """ Create table MLA_Rel as
      Select a.AC_Num, a.AC_Name,
      b.MLA_Name as MLA_2018, b.Party as Party_2018,
      c.MLA_Name as MLA_2023, c.Party as Party_2023
      from KA_Constituencies as a, MLA_2018 as b, MLA_2023 as c
      where a.AC_Num=b.AC_Num
      AND a.AC_Num=c.AC_Num """

con.execute(sql)
con.commit()

In [32]:
pd.read_sql("Select * from MLA_Rel",con)

,AC_Num,AC_Name,MLA_2018,Party_2018,MLA_2023,Party_2023
0,1,Nippani,Shashikala Annasaheb Jolle,BJP,Shashikala Annasaheb Jolle,BJP
1,2,Chikkodi-Sadalga,Ganesh Hukkeri,INC,Ganesh Prakash Hukkeri,Congress
2,3,Athani,Mahesh Kumathalli,BJP,Laxman Savadi,Congress
3,4,Kagwad,Srimant Patil,BJP,Bharamgouda Alagouda Kage,Congress
4,5,Kudachi,P. Rajeev,BJP,Mahendra Kallappa Tammannavar,Congress
...,...,...,...,...,...,...
219,220,T Narasipura,Ashvin Kumar M.,JDS,Dr H C Mahadevappa,Congress
220,221,Hanur,R. Narendra,INC,M R Manjunath,JDS
221,222,Kollegal,N. Mahesh,BJP,A R Krishnamurthy,Congress
222,223,Chamrajanagar,C. Puttarangashetty,INC,C Puttarangashetty,Congress


  5. Use tables KA_Constituencies, MLA_2018, and MLA_2023.  
        Using appropriate JOINs, print   
        AC_Num, AC_Name, MLA in 2018, Party, MLA in 2023, and Party


In [33]:
sql = """ Select a.AC_Num, a.AC_Name,
      b.MLA_Name as MLA_2018, b.Party as Party_2018,
      c.MLA_Name as MLA_2023, c.Party as Party_2023
      from KA_Constituencies as a
      JOIN MLA_2018 as b USING(AC_Num)
      JOIN MLA_2023 as c USING(AC_Num)
      """

pd.read_sql(sql,con)

,AC_Num,AC_Name,MLA_2018,Party_2018,MLA_2023,Party_2023
0,1,Nippani,Shashikala Annasaheb Jolle,BJP,Shashikala Annasaheb Jolle,BJP
1,2,Chikkodi-Sadalga,Ganesh Hukkeri,INC,Ganesh Prakash Hukkeri,Congress
2,3,Athani,Mahesh Kumathalli,BJP,Laxman Savadi,Congress
3,4,Kagwad,Srimant Patil,BJP,Bharamgouda Alagouda Kage,Congress
4,5,Kudachi,P. Rajeev,BJP,Mahendra Kallappa Tammannavar,Congress
...,...,...,...,...,...,...
219,220,T Narasipura,Ashvin Kumar M.,JDS,Dr H C Mahadevappa,Congress
220,221,Hanur,R. Narendra,INC,M R Manjunath,JDS
221,222,Kollegal,N. Mahesh,BJP,A R Krishnamurthy,Congress
222,223,Chamrajanagar,C. Puttarangashetty,INC,C Puttarangashetty,Congress


6. Create table MLA_Join and insert the above values


In [34]:
sql = """ Create table MLA_Join as
      Select a.AC_Num, a.AC_Name,
      b.MLA_Name as MLA_2018, b.Party as Party_2018,
      c.MLA_Name as MLA_2023, c.Party as Party_2023
      from KA_Constituencies as a
      JOIN MLA_2018 as b USING(AC_Num)
      JOIN MLA_2023 as c USING(AC_Num) """

con.execute(sql)
con.commit()

In [35]:
pd.read_sql("Select * from MLA_Join",con)

,AC_Num,AC_Name,MLA_2018,Party_2018,MLA_2023,Party_2023
0,1,Nippani,Shashikala Annasaheb Jolle,BJP,Shashikala Annasaheb Jolle,BJP
1,2,Chikkodi-Sadalga,Ganesh Hukkeri,INC,Ganesh Prakash Hukkeri,Congress
2,3,Athani,Mahesh Kumathalli,BJP,Laxman Savadi,Congress
3,4,Kagwad,Srimant Patil,BJP,Bharamgouda Alagouda Kage,Congress
4,5,Kudachi,P. Rajeev,BJP,Mahendra Kallappa Tammannavar,Congress
...,...,...,...,...,...,...
219,220,T Narasipura,Ashvin Kumar M.,JDS,Dr H C Mahadevappa,Congress
220,221,Hanur,R. Narendra,INC,M R Manjunath,JDS
221,222,Kollegal,N. Mahesh,BJP,A R Krishnamurthy,Congress
222,223,Chamrajanagar,C. Puttarangashetty,INC,C Puttarangashetty,Congress


7. Taking appropriate samples, demonstrate INNER, LEFT, and RIGHT joins of tables.

In [36]:
pd.read_sql("""
SELECT a.AC_Num, a.MLA_Name MLA_2018_Name, a.Party Party_2018, b.MLA_Name MLA_2023_Name, b.Party Party_2023
FROM MLA_2018 a
    INNER JOIN MLA_2023 b
WHERE a.AC_Num = b.AC_Num
""", con)

,AC_Num,MLA_2018_Name,Party_2018,MLA_2023_Name,Party_2023
0,1,Shashikala Annasaheb Jolle,BJP,Shashikala Annasaheb Jolle,BJP
1,2,Ganesh Hukkeri,INC,Ganesh Prakash Hukkeri,Congress
2,3,Mahesh Kumathalli,BJP,Laxman Savadi,Congress
3,4,Srimant Patil,BJP,Bharamgouda Alagouda Kage,Congress
4,5,P. Rajeev,BJP,Mahendra Kallappa Tammannavar,Congress
...,...,...,...,...,...
219,220,Ashvin Kumar M.,JDS,Dr H C Mahadevappa,Congress
220,221,R. Narendra,INC,M R Manjunath,JDS
221,222,N. Mahesh,BJP,A R Krishnamurthy,Congress
222,223,C. Puttarangashetty,INC,C Puttarangashetty,Congress


In [37]:
pd.read_sql("""
SELECT a.AC_Num, a.MLA_Name MLA_2018_Name, a.Party Party_2018, b.MLA_Name MLA_2023_Name, b.Party Party_2023
FROM MLA_2018 a
    LEFT JOIN MLA_2023 b
WHERE a.AC_Num = b.AC_Num
""", con)

,AC_Num,MLA_2018_Name,Party_2018,MLA_2023_Name,Party_2023
0,1,Shashikala Annasaheb Jolle,BJP,Shashikala Annasaheb Jolle,BJP
1,2,Ganesh Hukkeri,INC,Ganesh Prakash Hukkeri,Congress
2,3,Mahesh Kumathalli,BJP,Laxman Savadi,Congress
3,4,Srimant Patil,BJP,Bharamgouda Alagouda Kage,Congress
4,5,P. Rajeev,BJP,Mahendra Kallappa Tammannavar,Congress
...,...,...,...,...,...
219,220,Ashvin Kumar M.,JDS,Dr H C Mahadevappa,Congress
220,221,R. Narendra,INC,M R Manjunath,JDS
221,222,N. Mahesh,BJP,A R Krishnamurthy,Congress
222,223,C. Puttarangashetty,INC,C Puttarangashetty,Congress


In [38]:
pd.read_sql("""
SELECT a.AC_Num, a.MLA_Name AS MLA_2018_Name, a.Party AS Party_2018, b.MLA_Name AS MLA_2023_Name, b.Party AS Party_2023
FROM MLA_2018 a
LEFT JOIN MLA_2023 b
ON b.AC_Num = a.AC_Num
""", con)


,AC_Num,MLA_2018_Name,Party_2018,MLA_2023_Name,Party_2023
0,1,Shashikala Annasaheb Jolle,BJP,Shashikala Annasaheb Jolle,BJP
1,2,Ganesh Hukkeri,INC,Ganesh Prakash Hukkeri,Congress
2,3,Mahesh Kumathalli,BJP,Laxman Savadi,Congress
3,4,Srimant Patil,BJP,Bharamgouda Alagouda Kage,Congress
4,5,P. Rajeev,BJP,Mahendra Kallappa Tammannavar,Congress
...,...,...,...,...,...
219,220,Ashvin Kumar M.,JDS,Dr H C Mahadevappa,Congress
220,221,R. Narendra,INC,M R Manjunath,JDS
221,222,N. Mahesh,BJP,A R Krishnamurthy,Congress
222,223,C. Puttarangashetty,INC,C Puttarangashetty,Congress
